# ML Flow

# **Introduction**

MLflow is an open-source platform designed to manage the complete machine learning lifecycle.

it Help in the machine learning workflow, from experimentation to deployment. As a Python package, it can be easily installed using `pip install mlflow`. MLflow is composed of four main modules:

- **Tracking**:  allows you to log and query experiments, including code versions, parameters, metrics, and output files, ensuring you can reproduce and compare results across different runs.

- **Projects**: provides a standard format for packaging reusable data science code. It enables you to organize your code in a way that makes it easy to run and reproduce across different environments.

- **Models**: MLflow Models offers a unified approach for packaging machine learning models. It supports multiple formats (e.g., TensorFlow, PyTorch, Scikit-Learn) 

- **Registry**: The Model Registry provides a centralized place to manage and govern the full lifecycle of your models. It includes functionalities for versioning, staging, and annotating models, helping maintain an organized workflow from experimentation to production.


## **Tracking experiments**

MLflow traking module allows to organize the experiments into runs, and keep track of the :

- **Parameters**:  path to train dataset, number of epochs, learning rate, etc.

- **metrics**: accuracy, loss, etc.

- **metadata**: code version, git commit, etc.

- **artifcats**: any file produced during the run, such as model, plots, etc.

- **models**: the model itself, which can be saved and loaded using the MLflow API.



We first need to define a database to store the experiments. This can be done by setting the `MLFLOW_TRACKING_URI` environment variable. If not set, MLflow will use a local SQLite database.

```python


# Homework



The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and 
model management.


### Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?


- **mlflow, version 2.13.0**

### Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* **4**
* 7

In [23]:
%%bash

DEST_DIR="$(pwd)/data/raw"
mkdir -p $DEST_DIR

# Base URL for the files
BASE_URL="https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-"

# List of months to download
MONTHS=("01" "02" "03")

# Download each file
for MONTH in "${MONTHS[@]}"; do
    URL="${BASE_URL}${MONTH}.parquet"
    wget -P $DEST_DIR $URL
done


--2024-05-28 12:45:42--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2503:6600:b:20a5:b140:21, 2600:9000:2503:8400:b:20a5:b140:21, 2600:9000:2503:7400:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2503:6600:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1,4M) [binary/octet-stream]
Saving to: ‘/home/marcos/GitHub/mlops-zoomcamp/01-experiment-tracking/data/raw/green_tripdata_2023-01.parquet’

     0K .......... .......... .......... .......... ..........  3%  320K 4s
    50K .......... .......... .......... .......... ..........  7%  466K 3s
   100K .......... .......... .......... .......... .......... 10%  921K 3s
   150K .......... .......... .......... .......... .......... 14%  937K 2s
   200K .......... .......... .......... .......... .......... 17% 1,41

In [24]:
%%bash

DEST_DIR="$(pwd)/data/processed"
DATA_PATH="$(pwd)/data/raw"

python preprocess_data.py --raw_data_path $DATA_PATH --dest_path $DEST_DIR

ls $DEST_DIR

dv.pkl
test.pkl
train.pkl
val.pkl




### Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* **2**
* 4
* 8
* 10

To start the mlflow UI we can use

´´´bash
mlflow ui --backend-store-uri sqlite:///mlflow.db
´´´

In [20]:
!mlflow ui --backend-store-uri sqlite:///mlflow.db

2024/05/28 11:22:02 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/05/28 11:22:02 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [28]:
%%bash

DEST_DIR="$(pwd)/data/processed"
python train.py --data_path $DEST_DIR

2024/05/28 13:53:48 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.
2024/05/28 13:53:48 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/28 13:53:49 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 13:54:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/marcos/.local/share/virtualenvs/01-experiment-tracking-6ItmHOOv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


<center><img src="figures/mlflow-experiment.png"></center>


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* **`default-artifact-root`**
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`


launching the tracking server on local machine with SQLite db for the backend store:

```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./artifacts
```



## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* **5.335**
* 5.818
* 6.336


In [32]:
%%bash
DEST_DIR="$(pwd)/data/processed"
python hpo.py --data_path $DEST_DIR


100%|██████████| 15/15 [00:47<00:00,  3.17s/trial, best loss: 5.335419588556921]




## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [34]:
%%bash
DEST_DIR="$(pwd)/data/processed"
python register_model.py --data_path $DEST_DIR

2024/05/28 14:56:05 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/28 14:56:06 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 14:56:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/marcos/.local/share/virtualenvs/01-experiment-tracking-6ItmHOOv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Traceback (most recent call last):
  File "/home/marcos/GitHub/mlops-zoomcamp/01-experiment-tracking/register_model.py", line 88, in <module>
    run_register_model()
  File "/home/marcos/.local/share/virtualenvs/01-experiment-tracking-6ItmHOOv/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
 

CalledProcessError: Command 'b'DEST_DIR="$(pwd)/data/processed"\npython register_model.py --data_path $DEST_DIR\n'' returned non-zero exit status 1.